In [3]:
# using the unpacked dump of wikipedia, collect all links (surface forms) and the corresponding href page (entities)
# the whole process takes approx 1h 10'

import os, pprint, sys, time, pickle, json
from bs4 import BeautifulSoup as bs
root = "/data/cm/"
root_data = root+"data/"

In [2]:
def process_file(filepath):
    
    aliases = {}
    
    data = open(filepath, "r", encoding="utf-8").read()
    soup = bs(data)
    docs = soup.findAll('doc')
    
    for doc in docs:
        links = [a for a in doc.findAll('a') if a.has_attr('href')]
        for link in links:
            href = link['href']
            link_text = link.string
            if href not in aliases.keys():
                aliases[href] = set()
            aliases[href].add(link_text)
    
    return aliases

In [5]:
start_task = time.time()
for folder in os.listdir(root_data+"wiki/de/unpacked/"):
    start = time.time()
    print("Processing ", folder, end=" ")
    
    all_mentions = {} # ent_name: [sf1, sf2, ...]
    
    for file in os.listdir(root_data+"wiki/de/unpacked/"+folder+"/"):
        
        
    
        filepath = root_data+"wiki/de/unpacked/"+folder+"/"+file
    
        aliases = {}
    
        input_file = open(filepath, "r", encoding="utf-8")
        data = input_file.read()
        soup = bs(data)
        docs = soup.findAll('doc')
    
        for doc in docs:
            links = [a for a in doc.findAll('a') if a.has_attr('href') and not a['href'].startswith('http')]
            for link in links:
                href = link['href']
                link_text = link.string
                if href not in aliases.keys():
                    aliases[href] = set()
                aliases[href].add(link_text)
        input_file.close()
        
        # update global dictionary
        for ent_name, surface_forms in aliases.items():
            if ent_name not in all_mentions.keys():
                all_mentions[ent_name] = surface_forms
            else:
                all_mentions[ent_name].update(surface_forms)
    end = time.time()
    
    
    
    
    
    entity_to_sfs = {k: list(v) for k,v in all_mentions.items()}
    
    with open(root+'output/wiki/surface_form_dictionary/partials/'+folder+'.json', 'w', encoding='utf-8') as outfile:
        json.dump(entity_to_sfs, outfile, indent=2, ensure_ascii=False)
    
    
    print("...", str(round(end-start, 2)), '\t\t', sys.getsizeof(all_mentions)/1000000)
    break
end_task = time.time()
print("Total time", str(round(end_task-start_task, 2)))

Processing  CZ ... 43.55 		 10.485856
Total time 44.66


In [4]:
entity_to_sfs = {k: list(v) for k,v in all_mentions.items()}

In [5]:
sf_to_entities = {}
for entity, sfs in entity_to_sfs.items():
    for sf in sfs:
        if sf not in sf_to_entities.keys():
            sf_to_entities[sf] = set()
        sf_to_entities[sf].add(entity)
sf_to_entities = {k: list(v) for k,v in sf_to_entities.items()}